# PRUEBA TECNICA INGENIERO DE DATOS PRAGMA

In [1163]:
import pandas as pd
import numpy as np


### RUTAS ARCHIVOS CSV

In [1164]:
ruta_archivo_1 = './2012-1.csv'
ruta_archivo_2 = './2012-2.csv'
ruta_archivo_3 = './2012-3.csv'
ruta_archivo_4 = './2012-4.csv'
ruta_archivo_5 = './2012-5.csv'
ruta_archivo_6 = './validation.csv'

### PRE-PROCESAMIENTO

In [1165]:
def pre_procesamiento(ruta_archivo):
    # Leer archivo .CSV
    datos = pd.read_csv(ruta_archivo)
    # Convertir formato de fecha a tipo Datetime
    datos['timestamp'] = pd.to_datetime(datos['timestamp'],errors='coerce')
    # Limpiar los datos con datos vacios
    datos_clean = datos.dropna()
    # Convertir los datos a una lista
    lista_valores = datos_clean.values.tolist()
    
    return (lista_valores)

### CONEXIÓN BASE DE DATOS SQL

In [1166]:
import mysql.connector as connection

### CONSULTAR ULTIMO REGISTRO

In [1167]:
def consulta_ultimo_registro():

    try:
        mydb = connection.connect(host='localhost',database='Transacciones',user='root',passwd='12345')
        query = """SELECT * FROM transacciones.medida_datos
                ORDER BY Conteo DESC
                LIMIT 1"""
        cursor = mydb.cursor()
        cursor.execute(query)
        resultado = cursor.fetchall()
       
    except Exception as e:
        mydb.close()
        print(str(e))
    else:
         return resultado

### ALMACENAMIENTO DE PROCESAMIENTO DE LOS DATOS

In [1168]:
def almacenar_procesamiento(mediciones):

    mydb = connection.connect(host='localhost',database='Transacciones',user='root',passwd='12345')

    try:
        query = """INSERT INTO  medida_datos 
                (Conteo,Valor_medio,Valor_max,Valor_min) 
                VALUES (%s,%s,%s,%s)"""
        cursor = mydb.cursor()
        cursor.executemany(query,mediciones)
        mydb.commit()
        #print(cursor.fetchall())
    except Exception as e:
        mydb.close()
        print(str(e))

### PROCESAMIENTO DE LOS DATOS

In [1169]:
def procesamiento_datos(datos_clean):
    valor_max = max(datos_clean)
    valor_min = min(datos_clean)
    valor_medio = np.mean(datos_clean)
    count = len(datos_clean)
    #valor_medio = valor_sumatoria / count
    mediciones = [[count,valor_medio,valor_max,valor_min]]
    #print((datos_clean))
    #print((mediciones))
    almacenar_procesamiento(mediciones)
    

### ALMACENAMIENTO DE ARCHIVOS .CSV

In [1170]:
lista_registro =[]

In [1171]:
def almacenar_datos(lista_valores):
    try:
        mydb = connection.connect(host='localhost',database='Transacciones',user='root',passwd='12345')

        for i in range (0,len(lista_valores),1):
            query = """INSERT INTO  datos_2012 
                (Timestamp,Price,User_ID) 
                VALUES (%s,%s,%s)"""
            cursor = mydb.cursor()
            cursor.execute(query,lista_valores[i])
            mydb.commit()
            lista_registro.append(lista_valores[i][1])
            procesamiento_datos(lista_registro)
            
    except Exception as e:
        mydb.close()
        print(str(e))
    

## MAIN

In [1172]:

#Ejecucion archivo_1

datos_leido_1 = pre_procesamiento(ruta_archivo_1)
almacenar_datos(datos_leido_1)

#Ejecucion archivo_2

datos_leido_2 = pre_procesamiento(ruta_archivo_2)
almacenar_datos(datos_leido_2)

#Ejecucion archivo_3

datos_leido_3 = pre_procesamiento(ruta_archivo_3)
almacenar_datos(datos_leido_3)

#Ejecucion archivo_4

datos_leido_4 = pre_procesamiento(ruta_archivo_4)
almacenar_datos(datos_leido_4)

#Ejecucion archivo_5

datos_leido_5 = pre_procesamiento(ruta_archivo_5)
almacenar_datos(datos_leido_5)

#Ejecucion validation

datos_leido_6 = pre_procesamiento(ruta_archivo_6)
almacenar_datos(datos_leido_6)
 